In [1]:
# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import re
from unicodedata import normalize as norm
from tensorflow.keras import layers
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pickle


# Carregamento das Musicas 

In [2]:
funk = pd.read_csv('funk.csv')
bossa_nova = pd.read_csv('bossa_nova.csv')
gospel = pd.read_csv('gospel.csv')
sertanejo = pd.read_csv('sertanejo.csv')

In [3]:
funk['label'] = 'funk'
bossa_nova['label'] = 'bossa nova'
gospel['label'] = 'gospel'
sertanejo['label'] = 'sertanejo'

In [4]:
dataset = pd.concat([funk,bossa_nova,gospel,sertanejo])

# Análise da volumetria 

In [5]:
dataset.groupby('label').count().sort_values(by=['lyric'],ascending=True)

,lyric
label,
bossa nova,800
funk,800
gospel,800
sertanejo,800


# Pré - processamento

In [15]:
def text_cleaner(text):
    
    nltk_stopwords = stopwords.words('portuguese')

    collection_text = [ {"text" : text}]
    text = pd.DataFrame(collection_text)

    text['text'] = text['text'].astype('str')
    text['text'] = text['text'].str.lower()
    text['text'] = text['text'].str.replace('\n',' ')
    text['text'] = text['text'].str.replace('\r',' ')
    text['text'] = text['text'].apply(lambda x: norm('NFKD', x).encode('ascii', 'ignore').decode())
    text['text'] = text['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',x))
    text['text'] = text['text'].apply(lambda x: re.sub(r'\s+',' ',x))
    pat = r'\b(?:{})\b'.format('|'.join(nltk_stopwords))
    text['text'] = text['text'].str.replace(pat,'')
    text = text['text'].values[0]

    return text

In [16]:
dataset['TEXTO_CLEAN'] = dataset['lyric'].apply(lambda x: text_cleaner(x))

## Função para contar o número de tokens

In [17]:
def calculate_number_words(text):

    quantity_of_words = text.split(" ")

    quantity_of_words = [i for i in quantity_of_words if i!=""]

    quantity_of_words = len(quantity_of_words)

    return quantity_of_words


In [18]:
dataset["COUNT_TOKENS"] = dataset["TEXTO_CLEAN"].apply(lambda x: calculate_number_words(x))

In [19]:
dataset.groupby('label').mean()

,COUNT_TOKENS
label,
bossa nova,88.42125
funk,150.65250
gospel,102.59750
sertanejo,102.90000


## Função para contar o número de tokens diferentes

In [20]:
def calculate_number_diferent_words(text):

    quantity_of_diferent_words = text.split(" ")

    quantity_of_diferent_words = [i for i in quantity_of_diferent_words if i!=""]

    quantity_of_diferent_words = set(quantity_of_diferent_words)

    quantity_of_diferent_words = list(quantity_of_diferent_words)

    quantity_of_diferent_words = len(quantity_of_diferent_words)

    return quantity_of_diferent_words

In [21]:
dataset["COUNT_DIFERENT_TOKENS"] = dataset["TEXTO_CLEAN"].apply(lambda x: calculate_number_diferent_words(x))

In [22]:
dataset.groupby('label').mean()

,COUNT_TOKENS,COUNT_DIFERENT_TOKENS
label,,
bossa nova,88.42125,53.03250
funk,150.65250,65.67625
gospel,102.59750,51.30750
sertanejo,102.90000,54.72125


In [44]:
dataset

,lyric,label,TEXTO_CLEAN,COUNT_TOKENS,COUNT_DIFERENT_TOKENS
0,"\nMaria! Maria!\nOw, Maria!\nEstoy enamorado ...",funk,maria maria ow maria estoy enamorado ti mari...,214,28
1,"\n[Nego do Borel]\nVocê partiu meu coração, a...",funk,nego borel voce partiu coracao ai amor na...,221,89
2,\nHi! My name is Lan\nVou dar game over\nNo s...,funk,hi my name is lan vou dar game over cu novi...,264,100
3,\nÉ o novo hit do verão\nPra geral curtir\nEl...,funk,novo hit verao pra geral curtir joga bum...,60,42
4,\nÉ a flauta envolvente que mexe com a mente\...,funk,flauta envolvente mexe mente ta presen...,124,22
...,...,...,...,...,...
795,\nA luta chegou de repente\nE te colocou fren...,sertanejo,luta chegou repente colocou frente frent...,126,63
796,\nPara de mentir pra você mesmo\nSeu amor por...,sertanejo,mentir pra voce amor mim nao segredo a...,58,41
797,\nQuadros nunca esquecem e sempre contarão\nO...,sertanejo,quadros nunca esquecem sempre contarao aco...,120,39
798,\nVoltei era de madrugada e me assustei\nAs l...,sertanejo,voltei madrugada assustei luzes acesas ...,87,78


# Train test split

In [23]:
x_train, x_test, y_train, y_test = train_test_split(dataset['TEXTO_CLEAN'].values, dataset['label'].values, test_size=0.3)

# Aplicação do TF-IDF

In [24]:
tokenizer = Tokenizer(num_words=10000)

In [25]:
tokenizer.fit_on_texts(x_train)

In [26]:
x_train_converted = tokenizer.texts_to_matrix(x_train,mode='tfidf')

# Encode Label

In [27]:
encode_label = preprocessing.LabelEncoder()

In [28]:
encode_label.fit(y_train)

LabelEncoder()

In [29]:
y_train_converted = encode_label.transform(y_train)

In [30]:
y_test_converted = encode_label.transform(y_test)

# Criação do modelo

In [31]:
model = tf.keras.Sequential()
model.add(layers.Dense(10, activation='relu', input_dim=10000))
model.add(layers.Dense(4, activation='softmax'))

In [32]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model.fit(x_train_converted,y_train_converted,epochs=100,validation_split=0.2)

Train on 1792 samples, validate on 448 samples
Epoch 1/100
1792/1792 [==============================] - 1s 627us/sample - loss: 1.0612 - accuracy: 0.5938 - val_loss: 0.7166 - val_accuracy: 0.7768
Epoch 2/100
1792/1792 [==============================] - 0s 142us/sample - loss: 0.3227 - accuracy: 0.9425 - val_loss: 0.5343 - val_accuracy: 0.8304
Epoch 3/100
1792/1792 [==============================] - 0s 124us/sample - loss: 0.1248 - accuracy: 0.9844 - val_loss: 0.4716 - val_accuracy: 0.8326
Epoch 4/100
1792/1792 [==============================] - 0s 109us/sample - loss: 0.0721 - accuracy: 0.9950 - val_loss: 0.4652 - val_accuracy: 0.8415
Epoch 5/100
1792/1792 [==============================] - 0s 98us/sample - loss: 0.0451 - accuracy: 0.9961 - val_loss: 0.4682 - val_accuracy: 0.8371
Epoch 6/100
1792/1792 [==============================] - 0s 99us/sample - loss: 0.0281 - accuracy: 0.9972 - val_loss: 0.4687 - val_accuracy: 0.8371
Epoch 7/100
1792/1792 [==============================] - 0s 1

1792/1792 [==============================] - 0s 105us/sample - loss: 0.0046 - accuracy: 0.9983 - val_loss: 0.5819 - val_accuracy: 0.8393
Epoch 56/100
1792/1792 [==============================] - 0s 97us/sample - loss: 0.0040 - accuracy: 0.9989 - val_loss: 0.5832 - val_accuracy: 0.8371
Epoch 57/100
1792/1792 [==============================] - 0s 97us/sample - loss: 0.0053 - accuracy: 0.9983 - val_loss: 0.5951 - val_accuracy: 0.8415
Epoch 58/100
1792/1792 [==============================] - 0s 97us/sample - loss: 0.0064 - accuracy: 0.9983 - val_loss: 0.5917 - val_accuracy: 0.8460
Epoch 59/100
1792/1792 [==============================] - 0s 99us/sample - loss: 0.0047 - accuracy: 0.9978 - val_loss: 0.5940 - val_accuracy: 0.8393
Epoch 60/100
1792/1792 [==============================] - 0s 97us/sample - loss: 0.0026 - accuracy: 0.9983 - val_loss: 0.6108 - val_accuracy: 0.8348
Epoch 61/100
1792/1792 [==============================] - 0s 95us/sample - loss: 0.0069 - accuracy: 0.9978 - val_loss:

# Avaliação do modelo

In [34]:
x_teste_converted = tokenizer.texts_to_matrix(x_test,mode='tfidf')

In [35]:
predict = model.predict(x_teste_converted)

In [36]:
predict = [np.argmax(predict[i]) for i in range(len(predict))]

In [37]:
predict = encode_label.inverse_transform(predict)

In [38]:
len(x_teste_converted)

960

In [39]:
len(y_test)

960

In [40]:
from sklearn.metrics import classification_report

print(classification_report(y_test,predict))

              precision    recall  f1-score   support

  bossa nova       0.84      0.75      0.79       254
        funk       0.95      0.84      0.89       238
      gospel       0.88      0.93      0.91       242
   sertanejo       0.68      0.81      0.74       226

    accuracy                           0.83       960
   macro avg       0.84      0.83      0.83       960
weighted avg       0.84      0.83      0.83       960



# Persistência do modelo

In [41]:
model.save("model.h5")

# Persistência do Tokenizer

In [42]:
with open('tokenizer.pickle', 'wb') as handle:

    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Persistência do Label Encoder

In [43]:
with open('label_encoder.pickle', 'wb') as handle:

    pickle.dump(encode_label, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load Model

In [ ]:
model = keras.models.load_model('model.h5')

In [ ]:
model.summary()

# Load Tokenizer

In [ ]:
with open('tokenizer.pickle', 'rb') as handle:

    tokenizer = pickle.load(handle)

# Load Label Encoder

In [ ]:
with open('label_encoder.pickle', 'rb') as handle:

    encode_label = pickle.load(handle)

# Frases de teste

In [ ]:
funk1 = "Ela só quer curtir Ela só quer zuar Gosta de dançar Dançar, dançar Se o papo for balada\nNão deixa pra depois Solteira de carteirinha"
funk2 = "Eu te amava no tempo da escola Mas você não me dava atenção\nPedi uma chance, até duas Mas você só me disse não\nMas você só me disse não Mas você só me disse não Mas com o tempo eu parei e fiz essa canção\nVou marcar de te ver e não ir"

gospel1 = "Senhor, meu Deus, quando eu, maravilhado Contemplo a tua imensa criação A terra e o mar e o céu todo estrelado Me vêm falar da tua perfeição Então minh'alma canta a ti"
gospel2 = " A dor não mata se Deus está presente Só quem caminha com Ele entende o valor De uma lágrima derramada no altar da dor Enxugada pelas mãos do consolador Se Ele quiser, Ele ressuscita mortos\nEle faz o impossível Tudo porque Ele é Deus Mas se Ele não quer que aconteça do meu jeito Eu declaro que eu aceito\nE agora o milagre sou eu, sou eu O milagre sou eu, sou eu Eu não vou parar"

sertanejo1 = "\nVocê pode ficar com quem você quiser\nNão tem nada a ver\nEu não mando em você\nMas ainda choro\nE quando alguém comenta não quero saber\nMe preocupo e apesar dos pesares\nEu sempre quero te ver bem\nE ainda vou além\nEm uma relação\nSei que não vai ser fácil amar outro alguém"
sertanejo2 = '\nContando, são 126 cabides\nE no guarda-roupa, um grande espaço seu\nDeixado\nO que faço? Estou eu bem no meu canto\nFuturo levado meu\nRoubado\nTô negociando com a solidão\nTô tentando convencer que ela não fique, não\nSe vá\nTô eu negociando com a solidão\nSó me cobre esse juros no final da outra estação\nE se vá\nEu nem quis te ver de malas prontas\nLevando o que eu temia\nSuas roupas e minha alegria\nE eu nem quis te ver de malas prontas\nNós dois não mais existe'

bossa_nova1 = 'Um cantinho e um violão\nEste amor, uma canção\nPra fazer feliz a quem se ama\nMuita calma pra pensar\nE ter tempo pra sonhar\nDa janela vê-se o Corcovado\nO Redentor que lindo'
bossa_nova2 = 'Vai tua vida\nTeu caminho é de paz e amor\nA tua vida\nÉ uma linda canção de amor\nAbre os teus braços e canta\nA última esperança\nA esperança divina\nDe amar em paz\nSe todos fossem\nIguais a você\nQue maravilha viver\nUma canção pelo ar\nUma mulher'


# Criação do pipeline

In [ ]:
text = text_cleaner(bossa_nova1)

sample_converted = tokenizer.texts_to_matrix([text],mode='tfidf')

predict = model.predict(sample_converted)

predict = np.argmax(predict[0])

predict = encode_label.inverse_transform([predict])

predict

In [ ]:
te = TextExplainer(random_state=42)
te.fit("Senhor, meu Deus, quando eu, maravilhado Contemplo a tua imensa criação A terra e o mar e o céu todo estrelado Me vêm falar da tua perfeição Então minh'alma canta a ti", model.predict_proba)
te.show_prediction(target_names=twenty_train.target_names)

In [ ]:
explainer_tf = LimeTextExplainer(class_names=[0,1,2,3])

In [ ]:
funk1

In [ ]:
model.predict_proba

In [ ]:
exp = explainer_tf.explain_instance(funk1,  model.predict, num_features=4, top_labels=2)

In [ ]:
exp.show_in_notebook(text=train_df['text'][idx], labels=(0,1))